In [ ]:
############################################################################
# 日経225 センチメントデータ 

  #ライブラリーの設定
import pandas as pd
import talib as ta
import numpy as np

def initialize(ctx):
    # 設定
    ctx.logger.debug("initialize() called")
    ctx.configure(
      target="jp.stock.daily",
      channels={          # 利用チャンネル
        "jp.stock": {
          "symbols": [
            "jp.stock.9983", #ファーストリテイリング
            "jp.stock.9984", #ソフトバンク
            "jp.stock.6954", #ファナック
            "jp.stock.9433", #KDDI
            "jp.stock.8028", #ファミリーマート
            "jp.stock.8035", #東京エレクトロン
            "jp.stock.4543", #テルモ
            "jp.stock.6367", #ダイキン
            "jp.stock.6971", #京セラ
            "jp.stock.9735", #セコム
          ],
          "columns": [
            #"open_price_adj",    # 始値(株式分割調整後)
            #"high_price_adj",    # 高値(株式分割調整後)
            #"low_price_adj",     # 安値(株式分割調整後)
            "close_price",        # 終値
            "close_price_adj",    # 終値(株式分割調整後) 
            "volume_adj",         # 出来高
            "txn_volume",         # 売買代金
            "ns_sentiment",      # ニュースとSNSのネガポジデータだよ。
                                  # 1〜-1で1に近いほどポジティブな発言が多いよ。
          ]
        }
      }
    )

    def _my_Sentiment(data):
        ns_sentiment = data["ns_sentiment"].fillna(method="ffill")
        # ctx.logger.debug(ns_sentiment)

        ns_sentiment_m75 = ns_sentiment.rolling(window=75, center=False).mean()
        ns_sentiment_m25 = ns_sentiment.rolling(window=25, center=False).mean()
        #ネガポジの25日と５日移動平均出してるよ

        buy_sig = ns_sentiment_m25 > ns_sentiment_m75
        sell_sig = ns_sentiment_m25 < ns_sentiment_m75
       
        return {
            "buy:sig": buy_sig,
            "sell:sig": sell_sig,
            "ns_sentiment_m25": ns_sentiment_m25,
            "ns_sentiment_m75": ns_sentiment_m75
        }

    # シグナル登録
    ctx.regist_signal("my_Sentiment", _my_Sentiment)

def handle_signals(ctx, date, current):

 # 買いシグナル
    buy = current["buy:sig"].dropna()
    for (sym,val) in buy.items():
        sec = ctx.getSecurity(sym)
        sec.order_target_percent(0.2, comment="SIGNAL BUY")
        pass

 # 売りシグナル
    sell = current["sell:sig"].dropna()
    for (sym,val) in sell.items():
        sec = ctx.getSecurity(sym)
        sec.order_target_percent(0, comment="SIGNAL SELL")
        pass
